In [1]:
#importing pandas library and readinginsurance data set
import pandas as pd
df =pd.read_csv('insurance_pre.csv')

In [2]:
df
#from this data set age , sex,ami , children,smoker and all input charges is output

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [3]:
#smoker and sex is categorical column so using get_dummies function performaing one hot encoding to change it to number
df = pd.get_dummies(df,drop_first=True)
df

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [4]:
df.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [5]:
# splitting input and output column
x_independent = df[['age', 'bmi', 'children',  'sex_male', 'smoker_yes']]
y_depenedent = df[['charges']]

In [7]:
#splitting train and test set
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_independent,y_depenedent,test_size=0.30,random_state=0)

In [8]:
#converting into standard form
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train_s = sc.fit_transform(x_train) 
x_test_s = sc.transform(x_test)

In [10]:
#!Pip install lightgbm


  Obtaining dependency information for lightgbm from https://files.pythonhosted.org/packages/b3/f8/ee33e36194eb03a76eccf3adac3fba51f0e56fbd20609bb531659d48d3cb/lightgbm-4.1.0-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.3 MB 140.9 kB/s eta 0:00:10
   - -------------------------------------- 0.0/1.3 MB 140.9 kB/s eta 0:00:10
   - -------------------------------------- 0.1/1.3 MB 173.0 kB/s eta 0:00:08
   -- ------------------------------------- 0.1/1.3 MB 179.1 kB/s eta 0:00:07
   -- ------------------------------------- 0.1/1.3 MB 218.5 kB/s eta 0:00:06
   --- ------------------------------------ 0.1/1.3 MB 252

In [12]:
# create function
#LGboosting checking 
def LGBoost_checking(x_train,x_test,y_train,y_test,x_train_s,x_test_s):
    ''' input should be x_train , x_test,y_train y_test in standard form and acutal form
    this function will create LGboosting Regressor model using all possible hyper parameter in normal form and standard form
     and make a note of all combo r2 core
     return value is 2 data frame 1 st data frame will give all reading 
     2nd data frame will give which combo is working good that combo details     
      
    '''
    import lightgbm as ltb
    from sklearn.metrics import r2_score
    LGBoost_score_df=pd.DataFrame(columns=['std/normal','n_estimators','r_score'])
    n_estimators = [50,100,150,200,300]
    form=['Standard','Normal']
    for f in form:
        for i in n_estimators:
            LGBoost_reg=ltb.LGBMRegressor(n_estimators=i,random_state =0)
            if(f=='Standard'):
                LGBoost_reg.fit(x_train_s,y_train)
                y_pred = LGBoost_reg.predict(x_test_s)
            else:
                LGBoost_reg.fit(x_train,y_train)
                y_pred = LGBoost_reg.predict(x_test)                
            LGBoost_r_score = r2_score(y_test,y_pred)
            LGBoost_score_df.loc[len(LGBoost_score_df.index)] = [f, i , LGBoost_r_score]
            max_score= LGBoost_score_df[LGBoost_score_df['r_score']==LGBoost_score_df['r_score'].max()]
    return (LGBoost_score_df,max_score)   
            

In [13]:
#checking LGboosting model
LGBoost_all,LGBoost_max=LGBoost_checking(x_train,x_test,y_train,y_test,x_train_s,x_test_s)

C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000159 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 316
[LightGBM] [Info] Number of data points in the train set: 936, number of used features: 5
[LightGBM] [Info] Start training from score 13232.916456
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000143 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 316
[LightGBM] [Info] Number of data points in the train set: 936, number of used features: 5
[LightGBM] [Info] Start training from score 13232.916456
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000064 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 316
[LightGBM] [Info] Number of data points in the train set: 936, number of used features: 5
[LightGBM] [Info] Start trainin

C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000064 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 316
[LightGBM] [Info] Number of data points in the train set: 936, number of used features: 5
[LightGBM] [Info] Start training from score 13232.916456
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000071 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 316
[LightGBM] [Info] Number of data points in the train set: 936, number of used features: 5
[LightGBM] [Info] Start training from score 13232.916456
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000034 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 313
[LightGBM] [Info] Number of data points in the train set:

In [14]:
LGBoost_max

,std/normal,n_estimators,r_score
0,Standard,50,0.879738


In [15]:
LGBoost_all

,std/normal,n_estimators,r_score
0,Standard,50,0.879738
1,Standard,100,0.869932
2,Standard,150,0.862926
3,Standard,200,0.856986
4,Standard,300,0.847778
5,Normal,50,0.876603
6,Normal,100,0.866032
7,Normal,150,0.856540
8,Normal,200,0.849162
9,Normal,300,0.838905


In [19]:
# this will train LGboosting model with given x and y train field with given n estimators
def LGBoost_Model(x_train,y_train,n_estimators):
    import lightgbm as ltb
    LGBoost_reg=ltb.LGBMRegressor(n_estimators=n_estimators,random_state =0)          
    LGBoost_reg.fit(x_train,y_train)                         
    return(LGBoost_reg)
            

In [20]:
LGBoost_reg=LGBoost_Model(x_train_s,y_train,50)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000220 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 316
[LightGBM] [Info] Number of data points in the train set: 936, number of used features: 5
[LightGBM] [Info] Start training from score 13232.916456


C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [22]:
#as we have used standard form to train  the model input should be standardised ,so converting input into standard form
std_input = sc.transform([[19,27.900,0,0,1]])
#funcion to transform input to standard form


C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [23]:
result = LGBoost_reg.predict(std_input)

In [24]:
result

array([18000.32574599])